In [1]:
import sys
import sktime
import tqdm as tq
import xgboost as xgb
import matplotlib
import seaborn as sns
import sklearn as skl
import pandas as pd
import numpy as np

import matplotlib.pyplot as plt
from tqdm import tqdm
from sktime.forecasting.model_selection import temporal_train_test_split
from sktime.utils.plotting import plot_series
from xgboost import XGBRegressor

In [11]:
base_path = "/home/jinjinjara1022/"

weather_forecast_data_1 = pd.read_csv(base_path + "OBIC/data/기상예측데이터_1.csv")
jeju_power_market_day_ahead = pd.read_csv(base_path + "OBIC/data/제주전력시장_시장전기가격_하루전가격.csv")

In [12]:
weather_forecast_data_1 = weather_forecast_data_1[weather_forecast_data_1['location'] != 'location']
weather_forecast_data_1['ts'] =  pd.to_datetime(weather_forecast_data_1['ts'], unit='s')
weather_forecast_data_1['base_ts'] = pd.to_datetime(weather_forecast_data_1['base_ts'], unit='s')

columns_to_average = [
    'temp', 'real_feel_temp', 'wet_bulb_temp', 'dew_point', 
    'wind_dir', 'wind_spd', 'wind_gust_spd', 'rel_hum', 'vis', 
    'ceiling', 'uv_idx', 'precip_prob', 'rain_prob', 'snow_prob', 
    'ice_prob', 'total_liq', 'rain', 'snow', 'ice', 'cld_cvr'
]

weather_forecast_data_1[columns_to_average] = weather_forecast_data_1[columns_to_average].apply(pd.to_numeric, errors='coerce')


average_df = weather_forecast_data_1.groupby('ts', as_index=False)[columns_to_average].mean()
weather_df = average_df

/tmp/ipykernel_882043/1489643764.py:2: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  weather_forecast_data_1['ts'] =  pd.to_datetime(weather_forecast_data_1['ts'], unit='s')
/tmp/ipykernel_882043/1489643764.py:3: FutureWarning: The behavior of 'to_datetime' with 'unit' when parsing strings is deprecated. In a future version, strings will be parsed as datetime strings, matching the behavior without a 'unit'. To retain the old behavior, explicitly cast ints or floats to numeric type before calling to_datetime.
  weather_forecast_data_1['base_ts'] = pd.to_datetime(weather_forecast_data_1['base_ts'], unit='s')


In [13]:
ahead_df = jeju_power_market_day_ahead
ahead_df['ts'] = pd.to_datetime(ahead_df['ts'], unit='s')

In [14]:
df = pd.merge(weather_df, ahead_df, on='ts')

In [15]:
df['hour'] = df['ts'].dt.hour
# df['hour_sin'] = np.sin(2 * np.pi * df['hour'] / 24)
# df['hour_cos'] = np.cos(2 * np.pi * df['hour'] / 24)

df['month'] = df['ts'].dt.month
# df['month_sin'] = np.sin(2 * np.pi * df['month'] / 12)
# df['month_cos'] = np.cos(2 * np.pi * df['month'] / 12)

df['day_of_week'] = df['ts'].dt.dayofweek
# df['day_of_week_sin'] = np.sin(2 * np.pi * df['day_of_week'] / 7)
# df['day_of_week_cos'] = np.cos(2 * np.pi * df['day_of_week'] / 7)

#df = df.drop(['ts', 'hour', 'month', 'day_of_week'], axis=1)
df = df[[col for col in df.columns if col != '하루전가격(원/kWh)'] + ['하루전가격(원/kWh)']]

df.rename(columns={
    '하루전가격(원/kWh)': 'previous_price_kwh'
}, inplace=True)

df['previous_day_price'] = df['previous_price_kwh'].shift(24)
df['previous_2day_price'] = df['previous_price_kwh'].shift(48)
df['previous_3day_price'] = df['previous_price_kwh'].shift(72)

In [16]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5641 entries, 0 to 5640
Data columns (total 28 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   ts                   5641 non-null   datetime64[ns]
 1   temp                 5641 non-null   float64       
 2   real_feel_temp       5641 non-null   float64       
 3   wet_bulb_temp        5641 non-null   float64       
 4   dew_point            5641 non-null   float64       
 5   wind_dir             5641 non-null   float64       
 6   wind_spd             5641 non-null   float64       
 7   wind_gust_spd        5641 non-null   float64       
 8   rel_hum              5641 non-null   float64       
 9   vis                  5641 non-null   float64       
 10  ceiling              5641 non-null   float64       
 11  uv_idx               5641 non-null   float64       
 12  precip_prob          5641 non-null   float64       
 13  rain_prob            5641 non-nul

In [20]:
df[100:].iloc[0]

ts                     2024-03-05 19:00:00
temp                              9.938267
real_feel_temp                    3.580247
wet_bulb_temp                     8.580246
dew_point                         7.037039
wind_dir                             339.0
wind_spd                         35.226756
wind_gust_spd                    61.298133
rel_hum                          82.777778
vis                              14.126458
ceiling                        4809.066667
uv_idx                                 0.0
precip_prob                      10.444444
rain_prob                        10.444444
snow_prob                              0.0
ice_prob                               0.0
total_liq                         0.028222
rain                              0.028222
snow                                   0.0
ice                                    0.0
cld_cvr                          74.222222
hour                                    19
month                                    3
day_of_week

In [21]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification

model_name = 'distilbert-base-uncased'  # 사용할 모델 선택
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=1)  # 회귀 문제이므로 num_labels=1

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

/home/jinjinjara1022/.local/lib/python3.10/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
